# Request and Response in Workflows


**Executors** are where computation happens. When an executor emits a message, it flows along the edges to downstream executors which activate when their dependent data becomes available. 

Beyond the internal flow, executors can also pause and send requests outside the workflow to wait for responses which is useful for any operations that depend on external systems or humans.

Now that we understand how tool approvals work to control agent actions, let's explore how Agent Framework handles HITL interactions at the workflow level. In this section, we'll examine how executors can pause execution to request human input or approval, wait for responses, and then continue processing based on that feedback.

## Learning Objectives

By completing this section, you will:

- **Understand** the request-response pattern of Agent Framework
- **Build** a complete content generation system with human approval requirements
- **Implement** multi-executor workflows
- **Apply** these concepts by building your own HITL approval system


### Key Steps in Request/Response message handling

Executors have two primary mechanisms for request-response interactions:

1. Sending requests via `ctx.request_info`:

    `ctx.request_info(request_data=request_data, response_type=response_type)`

**Purpose:** Pause workflow execution and request external input

**What happens:**

- Workflow emits a `RequestInfoEvent` containing your request data
- Workflow execution pauses at this point
- The event surfaces to the external application
- External system processes the request and provides a response

2. Handling responses via `@response_handler`:

    ```python
    @response_handler
    async def on_human_feedback(
        self,
        feedback: str,
        ctx: WorkflowContext[AgentExecutorRequest, str],
    ) -> None:
        # Process response and send message to the next executor
    ```

**Purpose:** Define how to handle responses when they arrive

**What happens:**

- Automatically registered to handle responses matching the request/response types
- Framework routes the response to this method when received
- You can then continue processing or send messages to other executors

Executors can send requests directly using `ctx.request_info()` and handle responses using the `@response_handler` decorator


## Example - Content Generation with HITL

In this example, we'll build a content generation and approval workflow where AI-generated responses must receive human approval before being delivered to users. 

#### Key Components

```
User Query → Worker (AI Generator) → Reviewer (Escalator) → Human Moderator
                ↑                           ↓
                └──── Approval Decision ────┘
                              ↓
                        Final Output → User
```

The system consists of two executors operating in a feedback loop:

1. **Worker** generates AI responses and sends them for review
2. **Reviewer** escalates to a human for approval/rejection
3. **Human** makes the decision (simulated in this demo)
4. **Reviewer** forwards the decision back to Worker
5. **Worker** either emits approved content or regenerates based on feedback

---

### Setup

First, let's validate our environment and create the Azure OpenAI client.


In [ ]:
import sys
sys.path.insert(0, '..')

from workshop_utils import validate_env

validate_env()

In [ ]:
import os
from agent_framework.azure import AzureOpenAIChatClient

chat_client = AzureOpenAIChatClient(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
)

print("Client created successfully!")

In [ ]:
from dataclasses import dataclass
from typing import Optional
from enum import Enum
from uuid import uuid4


from agent_framework import ( 
    ChatMessage,
    Executor,
    AgentRunUpdateEvent,
    AgentResponseUpdate,
    Contents,
    FunctionCallContent,
    FunctionResultContent,
    Role,
    WorkflowAgent,
    WorkflowBuilder,
    ChatClientProtocol,
    WorkflowContext,
    RequestInfoEvent,
    handler,
    response_handler,
)

### Step 1 - Define Custom Messages

Custom dataclasses define **structured communication contracts** between executors in workflows. They enable type-safe message passing and allow handlers to route based on message type.

In our example we define three:
1. `ReviewRequest` - communication from Worker to Reviewer to send responses for evaluation
2. `ReviewResponse` - from Reviewer to Worker to deliver review decisions
3. `HumanReviewRequest`- communication from Reviewer to external system (human)

Handlers use `@handler` decorator with type hints to **auto-route** messages. 
For example, the `review` handler of `ReviewerWithHumanInTheLoop` only triggers when a ReviewRequest is received. 

In [ ]:
@dataclass
class ReviewRequest:
    """Structured request passed from Worker to Reviewer for evaluation."""

    request_id: str
    user_messages: list[ChatMessage]
    agent_messages: list[ChatMessage]

# Response data that comes after review
@dataclass
class ReviewResponse:
    """Structured response from Reviewer back to Worker."""

    request_id: str
    feedback: str
    approved: bool

@dataclass
class HumanReviewRequest:
    """A request message type for escalation to a human reviewer."""

    agent_request: ReviewRequest | None = None

### Step 2 - Add Executors

Understanding executor logic is crucial - we have two executors, `Worker` and `Reviewer`:

**`Worker`**

The Worker both performs LLM inference and manages the review lifecycle. It receives user queries, calls the LLM to generate responses, sends them for human review, and handles feedback by either emitting approved responses to users or regenerating improved versions based on reviewer comments.

Two handlers:
* `handle_user_messages` - entry point for user queries
  1. Builds `messages` conversation history: system prompts, user messages, agent responses
  2. Calls LLM to generate response
  3. Creates ReviewRequest with user messages and agent response
  4. Sends to Reviewer via `ctx.send_message()`
  5. Tracks `_pending_requests` mapping each request by ID to its data (ReviewRequest + associated messages)

- `handle_review_response`
  1. Processes each review response from Reviewer and checks for approvals 
  2. If Approved - emit `AgentRunUpdateEvent` with agent messages to deliver final response
  3. If Not Approved:
     1. Incorporate feedback and user messages to `messages`(conversation history)
     2. Retry calling LLM with updated context
     2. Create and send new ReviewRequest with the new agent response to the Reviewer

  4. Sends to Reviewer via `ctx.send_message()`
  5. Tracks `_pending_requests` mapping each request by ID to its data (ReviewRequest + associated messages)

In [ ]:
class Worker(Executor):
    """Executor that generates responses and incorporates feedback when necessary."""

    def __init__(self, id: str, chat_client: ChatClientProtocol) -> None:
        super().__init__(id=id)
        self._chat_client = chat_client
        self._pending_requests: dict[str, tuple[ReviewRequest, list[ChatMessage]]] = {}

    @handler
    async def handle_user_messages(self, user_messages: list[ChatMessage], ctx: WorkflowContext[ReviewRequest]) -> None:
        print("Worker: Received user messages, generating response...")

        # Initial context history setup
        messages = [ChatMessage(role=Role.SYSTEM, text="You are a helpful assistant. Be concise.")]
        messages.extend(user_messages)

        print("Worker: Calling LLM to generate response...")
        response = await self._chat_client.get_response(messages=messages)
        print(f"Worker: Response generated: {response.messages[-1].text}")

        # Add agent messages to context
        messages.extend(response.messages)

        # Create review request and send to Reviewer
        request = ReviewRequest(request_id=str(uuid4()), user_messages=user_messages, agent_messages=response.messages)
        print(f"Worker: Sending response for review (ID: {request.request_id[:8]})")
        await ctx.send_message(request)

        # Track request for possible retry.
        self._pending_requests[request.request_id] = (request, messages)

    @handler
    async def handle_review_response(self, review: ReviewResponse, ctx: WorkflowContext[ReviewRequest]) -> None:
        print(f"Worker: Received review for request {review.request_id[:8]} - Approved: {review.approved}")

        if review.request_id not in self._pending_requests:
            raise ValueError(f"Unknown request ID in review: {review.request_id}")
        
        request, messages = self._pending_requests.pop(review.request_id)

        if review.approved:
            print("Worker: Response approved. Emitting to external consumer...")
            contents: list[Contents] = []
            for message in request.agent_messages:
                contents.extend(message.contents)

            # Emit approved result to external consumer via AgentRunUpdateEvent
            await ctx.add_event(
                AgentRunUpdateEvent(self.id, data=AgentResponseUpdate(contents=contents, role=Role.ASSISTANT))
            )
            return

        print(f"Worker: Response not approved. Feedback: {review.feedback}")
        print("Worker: Regenerating response with feedback...")

        messages.append(ChatMessage(role=Role.SYSTEM, text=review.feedback))
        messages.append(
            ChatMessage(role=Role.SYSTEM, text="Please incorporate the feedback and regenerate the response.")
        )
        messages.extend(request.user_messages)

        response = await self._chat_client.get_response(messages=messages)
        print(f"Worker: New response generated: {response.messages[-1].text}")
        messages.extend(response.messages)

        # Send updated request for re-review.
        new_request = ReviewRequest(
            request_id=review.request_id, user_messages=request.user_messages, agent_messages=response.messages
        )
        await ctx.send_message(new_request)

        self._pending_requests[new_request.request_id] = (new_request, messages)

2. **`ReviewerWithHumanInTheLoop`**

The Reviewer acts as a human escalation point in the review process. It receives response proposals from the Worker, escalates them to a human for evaluation via `ctx.request_info()`, and forwards the human's decision back to the Worker.

**Handlers**:
- `review()`: Receives ReviewRequest, escalates to human via `ctx.request_info()` 
- `accept_human_review()`: Receives human's ReviewResponse, forwards to Worker using stored `worker_id`

In [ ]:
class ReviewerWithHumanInTheLoop(Executor):
    """Executor that always escalates reviews to a human manager."""

    # when we create an instance, the id is needed so we can send responses back to it
    def __init__(self, worker_id: str, reviewer_id: str | None = None) -> None:
        unique_id = reviewer_id or f"{worker_id}-reviewer"  # if a reviewer id is provided we use it and if not we create a custom ID
        super().__init__(id=unique_id)
        self._worker_id = worker_id

    # Handler 1 - receives the initial review request, runs when the Reviewer executor received a ReviewRequest message
    @handler
    async def review(self, request: ReviewRequest, ctx: WorkflowContext) -> None:

        print(f"Reviewer: Evaluating response for request {request.request_id[:8]}...")
        print("Reviewer: Escalating to human manager...")

        # Forward the request to a human manager by sending a HumanReviewRequest.
        await ctx.request_info(request_data=HumanReviewRequest(agent_request=request), response_type=ReviewResponse)

    # Handler 2 - receives the human response and 
    @response_handler
    async def accept_human_review(
        self,
        original_request: HumanReviewRequest,
        response: ReviewResponse,
        ctx: WorkflowContext[ReviewResponse],
    ) -> None:
        # Accept the human review response and forward it back to the Worker.
        print(f"Reviewer: Accepting human review for request {response.request_id[:8]}...")
        print(f"Reviewer: Human feedback: {response.feedback}")
        print(f"Reviewer: Human approved: {response.approved}")
        print("Reviewer: Forwarding human review back to worker...")
        await ctx.send_message(response, target_id=self._worker_id)

### Step 3 - Build the Workflow

Now we can finally build the workflow. 
Ours has **two executors** connected by **bidirectional edges** that enable a feedback loop. The Worker serves as the entry point, and the entire workflow is exposed as an agent interface.


**Executor Registration**:
We register two executors with named identifiers:

- Worker uses ID `"sub-worker"` for message routing and we also pass the chat client as this executor is responsible for LLM calls
- Reviewer stores `worker_id` to send responses back

**Communication Edges**:
- Worker sends ReviewRequests forward → Reviewer
- Reviewer sends ReviewResponses backward → Worker

In [ ]:
print("Building workflow with Worker-Reviewer cycle...")

agent = (
    WorkflowBuilder()
    .register_executor(
        lambda: Worker(
            id="sub-worker",
            chat_client=chat_client,
        ),
        name="worker",
    )
    .register_executor(
        lambda: ReviewerWithHumanInTheLoop(worker_id="sub-worker"),
        name="reviewer",
    )
    .add_edge("worker", "reviewer") 
    .add_edge("reviewer", "worker")
    .set_start_executor("worker")
    .build()
    .as_agent() 
)

### Step 4 - Workflow Execution

Now we're ready to run the workflow and process function calls. The key responsibilities in this step are:

1. **Execute the workflow** with the user query
2. **Detect human review requests** by screening for `FunctionCallContent` with name `REQUEST_INFO_FUNCTION_NAME`
3. **Parse the function call** to extract the `HumanReviewRequest` and its `request_id`
4. **Prepare the human response** (in production, this comes from actual human input - here it's pre-set)
5. **Resume the workflow** by sending the response wrapped in `FunctionResultContent` with matching `call_id`

It is important to match the function call with its result - the agent uses `call_id` to link responses back to their originating requests and resume execution at the right point.

In [ ]:
print("Running workflow agent with user query...")
print("Query: Write a Python function to calculate factorial")
print("-" * 50)

# Run the agent with an initial query
response = await agent.run("Write a Python function to calculate factorial")

# Locate the human review function call in the response messages
human_review_function_call: FunctionCallContent | None = None
for message in response.messages:
    for content in message.contents:
        if isinstance(content, FunctionCallContent) and content.name == WorkflowAgent.REQUEST_INFO_FUNCTION_NAME:
            human_review_function_call = content

# Handle the human review if required
if human_review_function_call:
    print("\n Workflow paused - Human review required")
    
    # Parse the function call arguments
    human_request_args = human_review_function_call.arguments
    request = (WorkflowAgent.RequestInfoFunctionArgs.from_json(human_request_args) 
               if isinstance(human_request_args, str) 
               else WorkflowAgent.RequestInfoFunctionArgs.from_dict(dict(human_request_args)))
    
    human_request = request.data
    if not isinstance(human_request, HumanReviewRequest):
        raise ValueError("Expected HumanReviewRequest")
    if human_request.agent_request is None:
        raise ValueError("Human review request must include agent_request.")

    request_id = human_request.agent_request.request_id

    # Simulate human approval for demonstration purposes.
    human_response = ReviewResponse(request_id=request_id, feedback="Approved", approved=True)

    # Create the function call result object to send back to the agent.
    human_review_function_result = FunctionResultContent(
        call_id=human_review_function_call.call_id,
        result=human_response,
    )
    # Send the human review result back to the agent.
    response = await agent.run(ChatMessage(role=Role.TOOL, contents=[human_review_function_result]))
    print(f"Final Agent Response: {response.messages[-1].text}")


print("=" * 50)
print("Workflow completed!")

**Understanding the Event-to-Function-Call Translation**

When the Reviewer calls `ctx.request_info()`, the workflow **emits a `RequestInfoEvent`** containing the `HumanReviewRequest` data. This event signals that the workflow needs external input.

**However**, when we call `await agent.run()`, we're interacting with the workflow **through the Agent interface** (`as_agent()`), not directly listening to internal events. The Agent interface translates internal workflow events into the standard message protocol.

**How does `HumanReviewRequest` Travels from Reviewer?**

When the Reviewer calls `ctx.request_info(request_data=HumanReviewRequest(...))`, the framework automatically serializes the custom dataclass and embeds it in a `FunctionCallContent` object. This function call is returned in the agent's response messages.

We then deserialize the function call arguments with `RequestInfoFunctionArgs.from_json()`, which gives us access to the original `HumanReviewRequest` via the `.data` attribute. The framework handles all serialization/deserialization - we just pass our custom dataclass in.

### Ready to Go

Now that we've covered the full request/response pattern, let's put this into practice by building our own workflow:

## Exercise - Create Your Own Workflow with HITL

In the following exercise, you'll create a system where AI generates draft responses to customer support tickets, and a **human supervisor must review and approve** every response before it's sent to customers.

The core workflow logic remains similar - generate content, request human review, handle feedback, but we'll add realistic context with ticket priorities, customer data, and supervisor decision-making.

**Your focus areas:**
- Understand how executors communicate
- Implement workflow request/response logic
- Build the workflow graph with proper edges
- Handle workflow events

### Workflow Architecture
```
Support Ticket → Intake → Draft Agent → Supervisor Review → Approved Response
                  (Entry)   (AI Writer)    (Human HITL)      (To Customer)
                                  ↑              ↓
                                  └──── Edits ───┘
```

Let's dive into the implementation!

#### Step 1 - Define Data Models

First, let's define the necessary data models. We've implemented the data models and enums that define the structure of our ticket system and workflow communication.

**Enums**
- `ReviewAction` - Supervisor's options: APPROVE or EDIT
- `TicketPriority` - Ticket urgency levels: LOW, MEDIUM, HIGH, CRITICAL

**Custom Message Types** (workflow communication):
- `SupportTicket` - Intake receives from external system
- `DraftRequest` - Intake → Draft Agent
- `ReviewRequest` - Draft Agent → Supervisor (contains draft response)
- `SupervisorDecision` - Supervisor → Draft Agent (after human review)

In [ ]:
class ReviewAction(Enum):
    """Actions a supervisor can take."""
    APPROVE = "approve"
    EDIT = "edit"

class TicketPriority(Enum):
    """Ticket priority levels."""
    LOW = "Low"
    MEDIUM = "Medium"
    HIGH = "High"
    CRITICAL = "Critical"

In [ ]:
@dataclass
class SupportTicket:
    """Represents a customer support ticket."""
    ticket_id: str
    customer_name: str
    subject: str
    description: str
    priority: TicketPriority = TicketPriority.MEDIUM

@dataclass
class DraftRequest:
    """Request from TicketIntake to DraftAgent (Worker)."""
    ticket: SupportTicket
    ticket_text: str

@dataclass
class ReviewRequest:
    """Request from DraftAgent (Worker) to Supervisor (Reviewer)."""
    ticket_id: str
    priority: str
    draft_response: str
    ticket: SupportTicket

@dataclass
class SupervisorDecision:
    """Supervisor's decision after reviewing the draft."""
    ticket_id: str
    action: ReviewAction
    modified_response: Optional[str]
    notes: str

#### Step 2 - Executors

Next, we've defined three executors that form the workflow pipeline. 

**Your task:** Review each executor and complete the TODOs.

The first one is **TicketIntakeExecutor**, the workflow entry point which receives the ticket and preprocesses it for the draft executor:

In [ ]:
class TicketIntakeExecutor(Executor):
    """Receives tickets and sends them to the DraftAgent."""
    
    def __init__(self):
        super().__init__(id="ticket-intake")
    
    @handler
    async def handle_ticket( self, ticket: SupportTicket, ctx: WorkflowContext[DraftRequest]
    ) -> None:
        """Process incoming ticket and send to DraftAgent."""

        print(f"Ticket Intake: {ticket.ticket_id} - {ticket.customer_name} ({ticket.priority.value})\n")
        print()
        
        # Prepare ticket text for AI
        ticket_text = f"""
            Ticket {ticket.ticket_id} - {ticket.customer_name})
            Priority: {ticket.priority.value} | Subject: {ticket.subject}
            Customer Message:
            {ticket.description}
            Please analyze this ticket and draft an appropriate response.
        """
        # TODO - Create a draft request in correct format and pass it to the next executor

The second one is **DraftAgentExecutor**, the AI worker which prompts the model and also handles final decision of the supervisor:

In [ ]:
class DraftAgentExecutor(Executor):
    """Generates AI draft responses and sends for supervisor review."""
    
    def __init__(self, chat_client):
        super().__init__(id="draft-agent")
        self._chat_client = chat_client
        self._pending_tickets: dict[str, SupportTicket] = {}
    
    @handler
    async def handle_draft_request(self, request: DraftRequest, ctx: WorkflowContext[ReviewRequest]
    ) -> None:
        """Generate AI draft and send for review."""
        print(f"Draft Agent: Generating response for {request.ticket.ticket_id}")
        
        INSTRUCTIONS = """
            You are an experienced customer support specialist. Your job is to:
            1. Analyze the support ticket
            2. Draft a professional, empathetic response

            For refund requests:
            - Acknowledge the customer's frustration
            - Explain the refund policy (14-day money-back guarantee)
            - Offer alternatives if applicable (troubleshooting, downgrade)
            - Be professional but empathetic

            Keep your response to 3-4 sentences. Be concise but helpful.
        """

        # TODO: Call the chat client with correct system and user chat messages

        # TODO: Extract the draft response text from the last message in response.messages

        # TODO: Store the ticket in _pending_tickets using ticket_id as key

        # TODO: Create a review request with correct values 

        # TODO: Send created review request to the Supervisor executor
        

    @handler
    async def handle_supervisor_decision(self, decision: SupervisorDecision, ctx: WorkflowContext
    ) -> None:
        """Handle supervisor's decision (approve/edit/escalate)."""

        print(f"Draft Agent: Received supervisor decision for {decision.ticket_id}")
        ticket = self._pending_tickets.pop(decision.ticket_id)

        if decision.action == ReviewAction.APPROVE:
            print(f"   ✓ Response approved - sent to {ticket.customer_name} | {decision.ticket_id} RESOLVED")
        elif decision.action == ReviewAction.EDIT:
            print(f"   ✓ Modified response - sent to {ticket.customer_name} | {decision.ticket_id} RESOLVED")

Finally, we'll build the **SupervisorReviewerExecutor**, the HITL reviewer which implements the core request/response pattern:

In [ ]:
class SupervisorReviewerExecutor(Executor):
    """Escalates reviews to human supervisor."""
    
    def __init__(self, draft_agent_id: str):
        super().__init__(id="supervisor-reviewer")
        self._draft_agent_id = draft_agent_id
        
    # TODO: Define a handler that receives ReviewRequest messages
    # TODO: Inside the handler, send a request to the external system with correct parameters

    # TODO: Define a response handler that receives the human supervisor decision
    # TODO: Inside the response handler, send the decision back to the Draft Agent with correct target id


<details>
<summary><strong>Click to reveal Step 2 solution</strong></summary>

```python

class TicketIntakeExecutor(Executor):
    """Receives tickets and sends them to the DraftAgent."""
    
    def __init__(self):
        super().__init__(id="ticket-intake")
    
    @handler
    async def handle_ticket( self, ticket: SupportTicket, ctx: WorkflowContext[DraftRequest]
    ) -> None:
        """Process incoming ticket and send to DraftAgent."""

        print(f"Ticket Intake: {ticket.ticket_id} - {ticket.customer_name} ({ticket.priority.value})\n")
        print()
        
        ticket_text = f"""
            Ticket {ticket.ticket_id} - {ticket.customer_name})
            Priority: {ticket.priority.value} | Subject: {ticket.subject}
            Customer Message:
            {ticket.description}
            Please analyze this ticket and draft an appropriate response.
        """
        
        draft_request = DraftRequest(ticket=ticket, ticket_text=ticket_text)
        await ctx.send_message(draft_request)


class DraftAgentExecutor(Executor):
    """Generates AI draft responses and sends for supervisor review."""
    
    def __init__(self, chat_client):
        super().__init__(id="draft-agent")
        self._chat_client = chat_client
        self._pending_tickets: dict[str, SupportTicket] = {}
    
    @handler
    async def handle_draft_request(self, request: DraftRequest, ctx: WorkflowContext[ReviewRequest]
    ) -> None:
        """Generate AI draft and send for review."""
        print(f"Draft Agent: Generating response for {request.ticket.ticket_id}")
        

        INSTRUCTIONS = """
            You are an experienced customer support specialist. Your job is to:
            1. Analyze the support ticket
            2. Draft a professional, empathetic response

            For refund requests:
            - Acknowledge the customer's frustration
            - Explain the refund policy (14-day money-back guarantee)
            - Offer alternatives if applicable (troubleshooting, downgrade)
            - Be professional but empathetic

            Keep your response to 3-4 sentences. Be concise but helpful.
        """
        
        response = await self._chat_client.get_response(
            messages=[
                ChatMessage(role=Role.SYSTEM, text=INSTRUCTIONS),
                ChatMessage(role=Role.USER, text=request.ticket_text)
            ]
        )
        draft_response = response.messages[-1].text

        self._pending_tickets[request.ticket.ticket_id] = request.ticket
        review_request = ReviewRequest(
            ticket_id=request.ticket.ticket_id,
            priority=request.ticket.priority.value,
            draft_response=draft_response,
            ticket=request.ticket
        )
        
        await ctx.send_message(review_request)

    @handler
    async def handle_supervisor_decision(self, decision: SupervisorDecision, ctx: WorkflowContext
    ) -> None:
        """Handle supervisor's decision (approve/edit/escalate)."""

        print(f"Draft Agent: Received supervisor decision for {decision.ticket_id}")
        ticket = self._pending_tickets.pop(decision.ticket_id)

        if decision.action == ReviewAction.APPROVE:
            print(f"   ✓ Response approved - sent to {ticket.customer_name} | {decision.ticket_id} RESOLVED")
        elif decision.action == ReviewAction.EDIT:
            print(f"   ✓ Modified response - sent to {ticket.customer_name} | {decision.ticket_id} RESOLVED")

    
class SupervisorReviewerExecutor(Executor):
    """Escalates reviews to human supervisor."""

    def __init__(self, draft_agent_id: str):
        super().__init__(id="supervisor-reviewer")
        self._draft_agent_id = draft_agent_id
        
    @handler
    async def review(self, request: ReviewRequest, ctx: WorkflowContext) -> None:
        print(f"Supervisor Reviewer: Escalating {request.ticket_id} to human\n")
        
        await ctx.request_info(
            request_data=request,
            response_type=SupervisorDecision
        )

    @response_handler
    async def accept_supervisor_decision(
        self,
        original_request: ReviewRequest, 
        response: SupervisorDecision,
        ctx: WorkflowContext[SupervisorDecision]
    ) -> None:
        print(f"Supervisor Reviewer: Forwarding decision to Draft Agent\n")
        await ctx.send_message(response, target_id=self._draft_agent_id)

```

</details>

#### Step 3 - Human Review Interface

Next, we've provided a console-based function that simulates supervisor review.

In a production system, this would be a web UI. For this exercise, we use simple console input to demonstrate the HITL pattern:

In [ ]:
def handle_supervisor_review_console(review_request: ReviewRequest) -> SupervisorDecision:
    """Console-based supervisor review."""

    print("\n" + "=" * 60)
    print(f"REVIEW: {review_request.ticket_id} | {review_request.priority}")
    print("=" * 60)
    print(review_request.draft_response)
    print("-" * 60)
    
    while True:
        choice = input("\n[1] Approve  [2] Edit: ").strip()
        
        if choice == "1":
            print("Response Approved\n")
            return SupervisorDecision(
                ticket_id=review_request.ticket_id,
                action=ReviewAction.APPROVE,
                modified_response=None,
                notes="Approved"
            )
        elif choice == "2":
            modified = input("Modified response: ").strip()
            print("Response Modified\n")
            return SupervisorDecision(
                ticket_id=review_request.ticket_id,
                action=ReviewAction.EDIT,
                modified_response=modified,
                notes="Modified by supervisor"
            )
        else:
            print("Invalid. Enter 1 or 2.")

#### Step 4 - Build the Workflow

Now assemble the workflow by connecting all three executors. You need to:

1. **Register three executors**
2. **Define three edges** (message flow paths)
3. **Set the entry point**
4. **Build** the workflow

In [ ]:
def build_workflow(chat_client):
    
    # TODO: Create a WorkflowBuilder and register all three executors
    # TODO: Add edges: intake→draft-agent, draft-agent→supervisor, supervisor→draft-agent
    # TODO: Set "intake" as the start executor
    # TODO: Call .build() to compile the workflow
    
    return workflow

<details>
<summary><strong>Click to reveal Step 4 solution</strong></summary>

```python

def build_workflow(chat_client):
    
    workflow = (
        WorkflowBuilder()
        .register_executor(
            lambda: TicketIntakeExecutor(),
            name="intake"
        )
        .register_executor(
            lambda: DraftAgentExecutor(chat_client),
            name="draft-agent"
        )
        .register_executor(
            lambda: SupervisorReviewerExecutor(draft_agent_id="draft-agent"),
            name="supervisor"
        )
        .add_edge("intake", "draft-agent")
        .add_edge("draft-agent", "supervisor")
        .add_edge("supervisor", "draft-agent")
        .set_start_executor("intake")
        .build()
    )
    
    return workflow
```

</details>

#### Step 5 - Run Workflow and Handle Events

Finally, execute the workflow and handle the human review pause point. The workflow will:
1. Run until it hits `ctx.request_info()` in the Supervisor
2. Pause and emit `RequestInfoEvent` objects
3. Wait for you to collect human decisions
4. Resume when you send responses back

In [ ]:
print("Human-in-the-Loop Workflow: Customer Support Ticket System")
print("=" * 70)
print()

workflow = build_workflow(chat_client)
sample_ticket = SupportTicket(
    ticket_id="TKT-78542",
    customer_name="Sarah Johnson",
    subject="Request for full refund on subscription",
    description=(
        "I signed up for the annual premium plan last week but the features don't work as advertised. "
        "The video conferencing keeps dropping and the file storage is extremely slow. "
        "I want a full refund and to cancel my subscription immediately."
    ),
    priority=TicketPriority.HIGH
)

print(f"Incoming: {sample_ticket.ticket_id} - {sample_ticket.customer_name}")
print(f"Subject: {sample_ticket.subject}\n")
print("Starting workflow...")

# TODO: Create an empty events list
# TODO: Stream events from workflow.run_stream(sample_ticket) and append each to events

# TODO: Filter events to find all RequestInfoEvent instances

# TODO: If there are request_info_events:
#       - Create an empty responses dict
#       - For each event, check if event data is a ReviewRequest
#       - Call handle_supervisor_review_console to get the supervisor's decision
#       - Store the decision in responses dict by id

# TODO: Resume the workflow by calling .send_responses_streaming() and append any new events to the events list

print(" Workflow completed!")

<details>
<summary><strong>Click to reveal Step 5 solution</strong></summary>

```python

print("Human-in-the-Loop Workflow: Customer Support Ticket System")
print("=" * 70)
print()

workflow = build_workflow(chat_client)
sample_ticket = SupportTicket(
    ticket_id="TKT-78542",
    customer_name="Sarah Johnson",
    subject="Request for full refund on subscription",
    description=(
        "I signed up for the annual premium plan last week but the features don't work as advertised. "
        "The video conferencing keeps dropping and the file storage is extremely slow. "
        "I want a full refund and to cancel my subscription immediately."
    ),
    priority=TicketPriority.HIGH
)

print(f"Incoming: {sample_ticket.ticket_id} - {sample_ticket.customer_name}")
print(f"Subject: {sample_ticket.subject}\n")
print("Starting workflow...")


events = []
async for event in workflow.run_stream(sample_ticket):
    events.append(event)

request_info_events = [
    event for event in events 
    if isinstance(event, RequestInfoEvent)
]

if request_info_events:
    print(f"  Pause - {len(request_info_events)} supervisor review(s) needed")

    responses = {}
    for event in request_info_events:
        if isinstance(event.data, ReviewRequest):
            decision = handle_supervisor_review_console(event.data)
            responses[event.request_id] = decision

    print("Resuming workflow with supervisor decision(s)...")
    async for event in workflow.send_responses_streaming(responses):
        events.append(event)


print(" Workflow completed!")

```

</details>

---
## Summary & Recap

In this notebook, you learned how to build **Human-in-the-Loop (HITL) workflows** that pause for external input and resume based on responses:

### Key Concepts

| Concept | Description |
|---------|-------------|
| **ctx.request_info()** | Pauses workflow execution and emits a `RequestInfoEvent` to request external input |
| **@response_handler** | Decorator that defines how to process responses when they arrive from external sources |
| **RequestInfoEvent** | Event emitted when workflow pauses - contains `request_id` to link responses back |
| **Type-Safe Routing** | Custom dataclasses + `@handler` decorators route messages based on type matching |

### What You Built

1. **Content Generation with HITL** - Simple Worker-Reviewer system with human approval
2. **Customer Support Ticket System** - Three-executor workflow with AI drafting and supervisor review
3. **Event Handling Pattern** - Detected pause points, collected human input, and resumed workflows

### Key Pattern
```python
# Pause for external input
await ctx.request_info(
    request_data=ReviewRequest(...),
    response_type=SupervisorDecision
)

# Handle the response
@response_handler
async def handle_response(self, original_request, response, ctx):
    await ctx.send_message(response, target_id=target_executor)

# Event handling
events = []
async for event in workflow.run_stream(data):
    events.append(event)

request_events = [e for e in events if isinstance(e, RequestInfoEvent)]
responses = {event.request_id: get_human_input(event.data) for event in request_events}

async for event in workflow.send_responses_streaming(responses):
    events.append(event)
```

### Moving Forward

You now understand how to build workflows that integrate human oversight for:
- Content approval and quality control
- High-stakes decisions requiring human judgment  
- Compliance scenarios where AI assists but humans approve

These patterns enable safe AI automation with appropriate human checkpoints.